In [ ]:
!lsusb

In [ ]:
from time import sleep
from datetime import datetime
from sh import gphoto2 as gp
import signal, os, subprocess
import os

In [ ]:
# Kill the gphoto process that starts whenever we turn on the camera or reboot the raspberry pi
def killGphoto2Process():
    p = subprocess.Popen(['ps', '-A'], stdout=subprocess.PIPE)
    out, err = p.communicate()

    # Search for the process we want to kill
    for line in out.splitlines():
        if b'gvfsd-gphoto2' in line:
            # Kill that process!
            pid = int(line.split(None,1)[0])
            os.kill(pid, signal.SIGKILL)

picID = "Eos5D"

clearCommand = ["--folder", "/store_00010001/DCIM/100EOS5D", \
                "--delete-all-files", "-R"]
triggerCommand = ["--trigger-capture"]
downloadCommand = ["--get-all-files"]

folder_name = picID
save_location = os.environ['HOME'] + "/Pictures/" + folder_name

def createSaveFolder():
    try:
        os.makedirs(save_location)
    except:
        print("Failed to create new directory.")
    os.chdir(save_location)

def captureImages():
    gp(triggerCommand)
    sleep(3)
    gp(downloadCommand)
    gp(clearCommand)
    renameFiles(picID)
    print(".", end="")

def renameFiles(ID):
    shot_time = datetime.now().strftime("%Y%m%d_%H%M%S_")
    for filename in os.listdir("."):
        if len(filename) < 13:
            os.rename(filename, (shot_time + ID + ".JPG"))

In [ ]:
killGphoto2Process()
createSaveFolder()

In [ ]:
captureImages()

In [ ]:
while True:
    captureImages()
    sleep(300)

In [ ]:
!zip -r -q images.zip images